In [2]:
import numpy as np
from scipy import stats
import pandas as pd

#models
from allennlp.predictors.predictor import Predictor
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from summarizer import Summarizer
bert_model = Summarizer()
predictor = Predictor.from_path("C:/Users/Silas_Dell/Downloads/Compressed/bidaf-elmo.2021-02-11.tar_2.gz")

# Suppressing unnwarranted warnings
import warnings
warnings.filterwarnings("ignore")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
error loading _jsonnet (this is expected on Windows), treating C:\Users\SILAS_~1\AppData\Local\Temp\tmp13z3vfwv\config.json as plain json


In [3]:
# Load Model and Tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [4]:
df = pd.read_csv('guardian_publications.csv')
df

,Unnamed: 0,title,authors,url,text,keywords,publish_date
0,0,Shell boss Ben van Beurden prepares to stand d...,Kalyeena Makortoff,https://www.theguardian.com/business/2022/sep/...,"Shell’s long-serving chief executive, Ben van ...","gas,beurden,privacy,say,profits,chief,shell,pr...",2022-09-02
1,1,South African court bans offshore oil and gas ...,NaN,https://www.theguardian.com/environment/2022/s...,A South African court has upheld a ban imposed...,"waves,gas,sq,judgment,shell,court,bans,south,m...",2022-09-01
2,2,European gas shortages likely to last several ...,Gwyn Topham,https://www.theguardian.com/business/2022/aug/...,Gas shortages across Europe are likely to last...,"wholesale,winters,gas,supplies,winter,chief,st...",2022-08-29
3,3,"Shell to pay £536,000 for overcharging prepaym...",Kalyeena Makortoff,https://www.theguardian.com/business/2022/aug/...,Shell has agreed to pay half a million pounds ...,"overcharging,privacy,536000,customers,shell,pr...",2022-08-25
4,4,Church of England pension boss’s shares in She...,Damian Carrington,https://www.theguardian.com/environment/2022/a...,The chair of the Church of England Pensions Bo...,"companies,pension,shocking,say,campaigners,cli...",2022-08-17
5,5,Shell to give staff 8% bonus after record profits,Julia Kollewe,https://www.theguardian.com/business/2022/aug/...,"Shell is handing nearly all its 82,000 staff a...","soaring,staff,profits,executive,shell,prices,b...",2022-08-03
6,6,Oil company profits boom as Americans reel fro...,Dominic Rushe,https://www.theguardian.com/business/2022/jul/...,The US’s biggest oil companies pumped out reco...,"gas,profits,boom,prices,fuel,reel,energy,quart...",2022-07-29
7,7,"With such eye-watering profits, Shell should i...",Nils Pratley,https://www.theguardian.com/business/nils-prat...,BP’s Bernard Looney put it more succinctly las...,"gas,halfyear,profits,shell,eyewatering,prices,...",2022-07-28
8,8,‘Insult to millions’: Shell and Centrica profi...,Rob Davies,https://www.theguardian.com/business/2022/jul/...,Soaring profits at two of the UK’s biggest ene...,"centrica,gas,profits,advice,shell,cause,prices...",2022-07-28
9,9,Biden says US economic slowdown due to Fed inf...,Graeme Wearden,https://www.theguardian.com/business/live/2022...,15.04 Biden: No surprise economy is slowing as...,"right,measures,inflationfighting,strong,biden,...",2022-07-28


In [5]:
df.rename(columns={'Field1':'questions'}, inplace=True)
df.rename(columns={'Field1.1':'text'}, inplace=True)
df

,Unnamed: 0,title,authors,url,text,keywords,publish_date
0,0,Shell boss Ben van Beurden prepares to stand d...,Kalyeena Makortoff,https://www.theguardian.com/business/2022/sep/...,"Shell’s long-serving chief executive, Ben van ...","gas,beurden,privacy,say,profits,chief,shell,pr...",2022-09-02
1,1,South African court bans offshore oil and gas ...,NaN,https://www.theguardian.com/environment/2022/s...,A South African court has upheld a ban imposed...,"waves,gas,sq,judgment,shell,court,bans,south,m...",2022-09-01
2,2,European gas shortages likely to last several ...,Gwyn Topham,https://www.theguardian.com/business/2022/aug/...,Gas shortages across Europe are likely to last...,"wholesale,winters,gas,supplies,winter,chief,st...",2022-08-29
3,3,"Shell to pay £536,000 for overcharging prepaym...",Kalyeena Makortoff,https://www.theguardian.com/business/2022/aug/...,Shell has agreed to pay half a million pounds ...,"overcharging,privacy,536000,customers,shell,pr...",2022-08-25
4,4,Church of England pension boss’s shares in She...,Damian Carrington,https://www.theguardian.com/environment/2022/a...,The chair of the Church of England Pensions Bo...,"companies,pension,shocking,say,campaigners,cli...",2022-08-17
5,5,Shell to give staff 8% bonus after record profits,Julia Kollewe,https://www.theguardian.com/business/2022/aug/...,"Shell is handing nearly all its 82,000 staff a...","soaring,staff,profits,executive,shell,prices,b...",2022-08-03
6,6,Oil company profits boom as Americans reel fro...,Dominic Rushe,https://www.theguardian.com/business/2022/jul/...,The US’s biggest oil companies pumped out reco...,"gas,profits,boom,prices,fuel,reel,energy,quart...",2022-07-29
7,7,"With such eye-watering profits, Shell should i...",Nils Pratley,https://www.theguardian.com/business/nils-prat...,BP’s Bernard Looney put it more succinctly las...,"gas,halfyear,profits,shell,eyewatering,prices,...",2022-07-28
8,8,‘Insult to millions’: Shell and Centrica profi...,Rob Davies,https://www.theguardian.com/business/2022/jul/...,Soaring profits at two of the UK’s biggest ene...,"centrica,gas,profits,advice,shell,cause,prices...",2022-07-28
9,9,Biden says US economic slowdown due to Fed inf...,Graeme Wearden,https://www.theguardian.com/business/live/2022...,15.04 Biden: No surprise economy is slowing as...,"right,measures,inflationfighting,strong,biden,...",2022-07-28


In [6]:
train = df.head(3)

In [7]:
# train.reset_index(drop=True, inplace=True)

In [8]:
df.head(3)

,Unnamed: 0,title,authors,url,text,keywords,publish_date
0,0,Shell boss Ben van Beurden prepares to stand d...,Kalyeena Makortoff,https://www.theguardian.com/business/2022/sep/...,"Shell’s long-serving chief executive, Ben van ...","gas,beurden,privacy,say,profits,chief,shell,pr...",2022-09-02
1,1,South African court bans offshore oil and gas ...,NaN,https://www.theguardian.com/environment/2022/s...,A South African court has upheld a ban imposed...,"waves,gas,sq,judgment,shell,court,bans,south,m...",2022-09-01
2,2,European gas shortages likely to last several ...,Gwyn Topham,https://www.theguardian.com/business/2022/aug/...,Gas shortages across Europe are likely to last...,"wholesale,winters,gas,supplies,winter,chief,st...",2022-08-29


In [9]:
result3 = []
for i in range(len(train["text"])):
    result3.append(len(tokenizer(train["text"][i])['input_ids']))
train["No.tokens"]=result3

Token indices sequence length is longer than the specified maximum sequence length for this model (698 > 512). Running this sequence through the model will result in indexing errors


In [11]:
resul = []
for i in range(len(train["No.tokens"])):
    # print()
    if train["No.tokens"][i] < 512:
        result = []
        for i in range(len(train["text"])):
            result.append(tokenizer.encode("summarize: "+train["text"][i], return_tensors='pt', 
                                        max_length=tokenizer.model_max_length, 
                                        truncation=True))
        train["tokens_input"]=result
        result1 = []
        for i in range(len(train["tokens_input"])):
            result1.append(model.generate(train["tokens_input"][i], min_length=80,
                                    max_length=150,
                                    length_penalty=20, 
                                    num_beams=2))
        train["summary_ids"]=result1                            
        result2 = []
        for i in range(len(train["summary_ids"])):
            result2.append(tokenizer.decode((train["summary_ids"][i])[0], skip_special_tokens=True))                            
        train["summary"]=result2

        
    else:
        res1 = []
        for i in range(len(train["text"])):
            res1.append(bert_model(train["text"][i], ratio=0.5))
        train["ext_summary"]=res1

        res2 = []
        for i in range(len(train["ext_summary"])):
            res2.append(tokenizer.encode("summarize: "+train["ext_summary"][i], return_tensors='pt', 
                                        max_length=tokenizer.model_max_length, 
                                        truncation=True))

        train["tokens_input_2"]=res2

        res3 = []
        for i in range(len(train["tokens_input_2"])):
            res3.append(model.generate(train["tokens_input_2"][i], min_length=80,
                                    max_length=150,
                                    length_penalty=20, 
                                    num_beams=2))

        train["summary_ids_2"]=res3

        result2 = []
        for i in range(len(train["summary_ids_2"])):
            result2.append(tokenizer.decode((train["summary_ids_2"][i])[0], skip_special_tokens=True))

        train["summary"]=result2

In [13]:
train[["text","No.tokens","summary",]]

,text,No.tokens,summary
0,"Shell’s long-serving chief executive, Ben van ...",698,"Ben van Beurden, who took over in 2014, would ..."
1,A South African court has upheld a ban imposed...,395,south african court upholds ban imposed on the...
2,Gas shortages across Europe are likely to last...,1022,cuts to supply of Russian gas since the invasi...


In [ ]:
train[["text","No.tokens","T5summary","summary_2"]].to_csv("gdb.csv")

In [ ]:
train

In [ ]:
result = []
for i in range(len(train["text"])):
    result.append(tokenizer.encode("summarize: "+train["text"][i], return_tensors='pt', 
                                max_length=tokenizer.model_max_length, 
                                truncation=True))
    
    

In [ ]:
train["tokens_input"]=result


In [ ]:
result1 = []
for i in range(len(train["tokens_input"])):
    result1.append(model.generate(train["tokens_input"][i], min_length=80,
                             max_length=150,
                             length_penalty=20, 
                             num_beams=2))

In [ ]:
train["summary_ids"]=result1


In [ ]:
result2 = []
for i in range(len(train["summary_ids"])):
    result2.append(tokenizer.decode((train["summary_ids"][i])[0], skip_special_tokens=True))

In [ ]:
result3 = []
for i in range(len(train["text"])):
    result3.append(len(tokenizer(train["text"][i])['input_ids']))

In [ ]:
train["T5summary"]=result2

train[["questions","text","No.tokens","T5summary"]]

In [ ]:
res1 = []
for i in range(len(train["text"])):
    res1.append(bert_model(train["text"][i], ratio=0.5))
train["ext_summary"]=res1


In [ ]:
res2 = []
for i in range(len(train["ext_summary"])):
    res2.append(tokenizer.encode("summarize: "+train["ext_summary"][i], return_tensors='pt', 
                                max_length=tokenizer.model_max_length, 
                                truncation=True))

train["tokens_input_2"]=res2

In [ ]:
res3 = []
for i in range(len(train["tokens_input_2"])):
    res3.append(model.generate(train["tokens_input_2"][i], min_length=80,
                             max_length=150,
                             length_penalty=20, 
                             num_beams=2))

train["summary_ids_2"]=res3


In [ ]:
res4 = []
for i in range(len(train["summary_ids_2"])):
    res4.append(tokenizer.decode((train["summary_ids_2"][i])[0], skip_special_tokens=True))

train["summary_2"]=res4


In [ ]:
train = train[["questions","text","No.tokens","T5summary","summary_2"]]

In [ ]:
train.to_csv("wdb.csv")